In [1]:
%matplotlib inline
%pylab inline
import os
from tqdm import tqdm, tqdm_notebook
from tqdm import trange

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import pylab as pl

from Bio import SeqIO

from mhcflurry import predict
from mhcflurry import class1_allele_specific

Populating the interactive namespace from numpy and matplotlib


Using Theano backend.
/home/vts/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Loading files:

In [2]:
record_dict = SeqIO.index("data/UP000005640_9606.fasta", "fasta")
# record_dict = SeqIO.to_dict(SeqIO.parse("data/uniprot_sprot.fasta", "fasta"))

In [3]:
list(record_dict.keys())[:9]

['tr|A0A024R161|A0A024R161_HUMAN',
 'tr|A0A075B6F4|A0A075B6F4_HUMAN',
 'tr|A0A075B6H5|A0A075B6H5_HUMAN',
 'tr|A0A075B6H7|A0A075B6H7_HUMAN',
 'tr|A0A075B6H8|A0A075B6H8_HUMAN',
 'sp|A0A075B6H9|LV469_HUMAN',
 'sp|A0A075B6I0|LV861_HUMAN',
 'sp|A0A075B6I1|LV460_HUMAN',
 'tr|A0A075B6I3|A0A075B6I3_HUMAN']

In [4]:
i = 2
len(record_dict['tr|A0A024R161|A0A024R161_HUMAN'].seq[i:i+9])

9

In [5]:
filename = 'output/peptides_raw.txt'
target = open(filename, 'w')

In [6]:
for key in tqdm(record_dict):
    lines = []
    for i in range(len(record_dict[key].seq)-9):
        lines.append(str(record_dict[key].seq[i:i+9]))
    for i in range(len(lines)):
        target.write(lines[i])
        target.write('\n')

100%|██████████| 21033/21033 [11:02<00:00, 31.76it/s]  


In [7]:
target.close()

In [8]:
df = pd.read_csv('output/peptides_raw.txt', index_col=False)
df.shape

(11314571, 1)

In [9]:
! sort output/peptides_raw.txt | uniq > output/human_peptides.txt

In [2]:
df = pd.read_csv('output/human_peptides.txt', index_col=False)
df.columns = ['Peptide']
df.shape

(10379345, 1)

In [ ]:
file1 = 'output/HLA-A0201.csv'
file2 = 'output/human_peptides.txt'
exceptions = 0
with open(file1, 'w') as output:
    output.write('Allele,Peptide,Prediction')
    output.write('\n')
    with open(file2, 'r') as file:
        for line in tqdm_notebook(file):
            s = file.readline()
            s = s.rstrip()
            try:
                prediction = predict(alleles=['HLA-A0201'], peptides=[s])
                output.write('{0},{1},{2}'.format(prediction.Allele.iloc[0], \
                                                prediction.Peptide.iloc[0], prediction.Prediction.iloc[0]))
                output.write('\n')
            except KeyError:
                exceptions += 1
                
print('{} exceptions occured'.format(exceptions))

In [49]:
df.iloc[139801]

Peptide    AEIPHRSGK
Name: 139801, dtype: object